### Reducing Data

This script will regrid the era-i and era-5 data and save it in research folder

In [5]:
import xarray as xr
import pandas as pd
import numpy as np
import xesmf as xe
import netCDF4

In [6]:
#Opening Climo Temperature and Standard Deviation Temp data from ERA-I 
ct_ds = xr.open_dataset("/erai/climo/mean/t.mean.climo.nc")
stdv_t_ds = xr.open_dataset('/erai/climo/stdv/t.stdv.climo.nc')

#Reading in ERA-5 Temperature Data
real1_ds = xr.open_dataset('/langlab_rit/andrea/ERA5data/era5_202101_t.nc')
real2_ds = xr.open_dataset('/langlab_rit/andrea/ERA5data/era5_202102_t.nc')

In [7]:
#Regridding ERA-I data using xesmf
ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(-90, 91.5, 1.5)),
        "lon": (["lon"], np.arange(0, 360, 1.5)),
        
    }
)

regridder = xe.Regridder(ct_ds, ds_out, "bilinear")
regridder

xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_256x512_121x240.nc 
Reuse pre-computed weights? False 
Input grid shape:           (256, 512) 
Output grid shape:          (121, 240) 
Periodic in longitude?      False

In [8]:
#Regridding ERA-5 data using xesmf
ds_out = xr.Dataset(
    {
        "lon": (["lon"], np.arange(0, 360, 1.5)),
        "lat": (["lat"], np.arange(-90, 91.5, 1.5)),  
    }
)

regridder2 = xe.Regridder(real1_ds, ds_out, "bilinear")
regridder2

xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_401x1440_121x240.nc 
Reuse pre-computed weights? False 
Input grid shape:           (401, 1440) 
Output grid shape:          (121, 240) 
Periodic in longitude?      False

In [9]:
ct_rgrd = regridder(ct_ds)
ct_rgrd

<xarray.Dataset>
Dimensions:  (time: 366, hour: 4, lev: 37, lat: 121, lon: 240)
Coordinates:
  * time     (time) float64 1.0 2.0 3.0 4.0 5.0 ... 363.0 364.0 365.0 366.0
  * hour     (hour) float64 0.0 6.0 12.0 18.0
  * lev      (lev) int32 1000 975 950 925 900 875 850 825 ... 30 20 10 7 5 3 2 1
  * lat      (lat) float64 -90.0 -88.5 -87.0 -85.5 -84.0 ... 85.5 87.0 88.5 90.0
  * lon      (lon) float64 0.0 1.5 3.0 4.5 6.0 ... 352.5 354.0 355.5 357.0 358.5
Data variables:
    t        (time, hour, lev, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes:
    regrid_method:  bilinear

In [10]:
stdvt_rgrd = regridder(stdv_t_ds)
stdvt_rgrd

<xarray.Dataset>
Dimensions:  (time: 366, hour: 4, lev: 37, lat: 121, lon: 240)
Coordinates:
  * time     (time) float64 1.0 2.0 3.0 4.0 5.0 ... 363.0 364.0 365.0 366.0
  * hour     (hour) float64 0.0 6.0 12.0 18.0
  * lev      (lev) int32 1000 975 950 925 900 875 850 825 ... 30 20 10 7 5 3 2 1
  * lat      (lat) float64 -90.0 -88.5 -87.0 -85.5 -84.0 ... 85.5 87.0 88.5 90.0
  * lon      (lon) float64 0.0 1.5 3.0 4.5 6.0 ... 352.5 354.0 355.5 357.0 358.5
Data variables:
    t        (time, hour, lev, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes:
    regrid_method:  bilinear

In [11]:
real1_rgrd = regridder2(real1_ds)
real1_rgrd

<xarray.Dataset>
Dimensions:  (time: 124, level: 19, lat: 121, lon: 240)
Coordinates:
  * level    (level) int32 1 3 5 7 10 30 50 70 ... 300 400 500 700 850 925 1000
  * time     (time) datetime64[ns] 2021-01-01 ... 2021-01-31T18:00:00
  * lon      (lon) float64 0.0 1.5 3.0 4.5 6.0 ... 352.5 354.0 355.5 357.0 358.5
  * lat      (lat) float64 -90.0 -88.5 -87.0 -85.5 -84.0 ... 85.5 87.0 88.5 90.0
Data variables:
    t        (time, level, lat, lon) float32 0.0 0.0 0.0 ... 242.3 242.3 242.3
Attributes:
    regrid_method:  bilinear

In [12]:
real2_rgrd = regridder2(real2_ds)
real2_rgrd

<xarray.Dataset>
Dimensions:  (time: 112, level: 19, lat: 121, lon: 240)
Coordinates:
  * level    (level) int32 1 3 5 7 10 30 50 70 ... 300 400 500 700 850 925 1000
  * time     (time) datetime64[ns] 2021-02-01 ... 2021-02-28T18:00:00
  * lon      (lon) float64 0.0 1.5 3.0 4.5 6.0 ... 352.5 354.0 355.5 357.0 358.5
  * lat      (lat) float64 -90.0 -88.5 -87.0 -85.5 -84.0 ... 85.5 87.0 88.5 90.0
Data variables:
    t        (time, level, lat, lon) float32 0.0 0.0 0.0 ... 250.8 250.8 250.8
Attributes:
    regrid_method:  bilinear

In [13]:
#Writing the ERA-I data into a netCDF in research folder
ct_rgrd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/regrid_data/ERAI_ct_rgrd.nc",
                               mode='w')
stdvt_rgrd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/regrid_data/ERAI_stdvt_rgrd.nc",
                               mode='w')

In [14]:
#Writing the ERA5 data into a netCDF in research folder
real1_rgrd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/regrid_data/ERA5_jant_rgrd.nc",
                               mode='w')
real2_rgrd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/regrid_data/ERA5_febt_rgrd.nc",
                               mode='w')